In [25]:
import joblib
from sklearn import linear_model
import os

In [12]:
np.array(list(train_df_x.columns)).shape

(15,)

In [19]:
#### TESTING THE IMBLEARN #####
import pandas as pd
import numpy as np
df_fold = pd.read_csv("../input/train_label_endoing.csv")

# Let's tale into the first fold only
train_df = df_fold[df_fold.kfold!=0]
valid_df = df_fold[df_fold.kfold==0]

# Check the imbalance in the data for now
print(train_df.MULTIPLE_OFFENSE.value_counts())

# print(train_df.columns)

train_df_x = train_df.drop(['DATE','INCIDENT_ID','MULTIPLE_OFFENSE','kfold'],axis=1).reset_index(drop=True)
print(f"train_df_x shape is : {train_df_x.shape}")
train_df_y = train_df.MULTIPLE_OFFENSE
# train_df_y = pd.DataFrame({'MULTIPLE_OFFENSE' : train_df['MULTIPLE_OFFENSE']})

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X,y = ros.fit_resample(train_df_x, train_df_y)

train_df = pd.concat([X,y], axis=1)

# Check the distribution now
print(train_df.MULTIPLE_OFFENSE.value_counts())

1    18230
0      854
Name: MULTIPLE_OFFENSE, dtype: int64
train_df_x shape is : (19084, 15)
1    18230
0    18230
Name: MULTIPLE_OFFENSE, dtype: int64


In [7]:
import pandas as pd
# When I ran the model, it was somewhat concerning as all the classifications have been of a single class. Therefore, we will have to look into it,

# Let's analyse the folds that we made, perhaps there was something fishy in the Stratified K-Fold sampling that we have done
df_fold = pd.read_csv("../input/train_folds.csv")
df_fold.head()
print(df_fold.shape)

(23856, 19)


In [6]:
for i in range(5):
    print(f"Current fold : {i}")
    print(df_fold[df_fold['kfold']==i].MULTIPLE_OFFENSE.value_counts())

Current fold : 0
1    4558
0     214
Name: MULTIPLE_OFFENSE, dtype: int64
Current fold : 1
1    4558
0     213
Name: MULTIPLE_OFFENSE, dtype: int64
Current fold : 2
1    4558
0     213
Name: MULTIPLE_OFFENSE, dtype: int64
Current fold : 3
1    4557
0     214
Name: MULTIPLE_OFFENSE, dtype: int64
Current fold : 4
1    4557
0     214
Name: MULTIPLE_OFFENSE, dtype: int64


In [4]:
# Therefore the stratified sampling technique looks on point, but three is a sever class imbalance, maybe we can look into some upsampling techniques to improve the quality of data

In [5]:
!pwd

/home/saransh/Programming/Hiring_Challenge_Novartis/Dataset/notebooks


In [ ]:
df = pd.read_csv("")

In [40]:
def get_feature_importance(model_name, model_type):
    l = os.listdir(f"../models/{model_name}/")
    all_clf = {}
    for f in l:
        if f.find(".pkl")!=-1:
            name = f[:f.find(".pkl")]
            all_clf[name] = joblib.load(f"../models/{model_name}/{f}")
    feautre_importance = {}
    for fold in range(5):
        if model_type=='linear':
            l = list(all_clf[f"fold_{fold}"].coef_[0])
        elif model_type=='tree':
            l = all_clf[f"fold_{fold}"].feature_importances_
        else:
            raise Exception("Not implemented")
        new_l = []
        for (i,val) in enumerate(l):
            curr_tup = (f"X_{i+1}",val)
            new_l.append(curr_tup)
        new_l.sort(key= lambda x : x[1], reverse=True)
        feautre_importance[f"fold_{fold}"] = new_l
    return feautre_importance

In [41]:
# feautre_importance_1e_minus_6 = get_feature_importance("logistic_regression_c_1e-6_label_encoder")
feature_importance_decision_trees = get_feature_importance("decision_tree_label_encoder", model_type="tree")

In [42]:
feature_importance_decision_trees

{'fold_0': [('X_10', 0.6235456405129233),
  ('X_15', 0.18588508216124405),
  ('X_11', 0.15978200543955579),
  ('X_12', 0.01463285779593148),
  ('X_4', 0.0040536165319473745),
  ('X_6', 0.0030145988288831317),
  ('X_2', 0.0022399821480747592),
  ('X_7', 0.0019949824521285444),
  ('X_8', 0.0009131786487371007),
  ('X_9', 0.0008941079938577485),
  ('X_14', 0.0008351919467216522),
  ('X_3', 0.0008293042487241414),
  ('X_13', 0.0007810565940686994),
  ('X_5', 0.00038509128454306674),
  ('X_1', 0.00021330341265914546)],
 'fold_1': [('X_10', 0.6176375937226721),
  ('X_11', 0.16152356891337977),
  ('X_12', 0.12229115762271851),
  ('X_15', 0.08657999982347803),
  ('X_6', 0.003996913557694143),
  ('X_4', 0.003722275086535396),
  ('X_7', 0.0014204707101249637),
  ('X_14', 0.0007910634035203823),
  ('X_2', 0.0007535294627132834),
  ('X_3', 0.0004435479926545746),
  ('X_13', 0.00032090061933364675),
  ('X_1', 0.0001935938134656933),
  ('X_5', 0.0001113658741256809),
  ('X_9', 0.00010744467773427283

It is a mutual consensus in the models that X_10 is the most relevant features are as follows: <br>
1) X_10 <br>
2) X_12 <br>
3) X_3 <br>
4) X_15 <br>
5) X_8 <br>
6) X_2 <br>
7) X_14 <br>
8) X_5 / X_9 <br>
<p>
Till the 7) point, the values the feature importance is in this very order

In [29]:
feautre_importance_1e_2 = get_feature_importance("logistic_regression_c_1e2_label_encoder")

In [30]:
feautre_importance_1e_2

{'fold_0': [('X_10', -0.5467936023563545),
  ('X_3', -0.07611808956108321),
  ('X_11', 0.0005285659439529121),
  ('X_7', 0.008510933100943127),
  ('X_14', 0.008641345115653218),
  ('X_13', 0.009105210133295058),
  ('X_5', 0.013559110145976959),
  ('X_8', 0.020228469722725538),
  ('X_15', 0.035932648795291636),
  ('X_4', 0.045476993648164044),
  ('X_6', 0.06546583327889302),
  ('X_2', 0.08299224078115794),
  ('X_9', 0.24636510009066115),
  ('X_1', 0.2816575845419582),
  ('X_12', 0.3632192293853987)],
 'fold_1': [('X_10', -0.5323255977142577),
  ('X_3', -0.03895249695761881),
  ('X_15', -0.00630728138518147),
  ('X_11', -0.00015015887715144887),
  ('X_14', 0.004642697473173597),
  ('X_13', 0.006923216376830237),
  ('X_7', 0.012106727035604621),
  ('X_6', 0.02685737639192136),
  ('X_8', 0.028383032221383053),
  ('X_2', 0.028501570361788717),
  ('X_5', 0.03843525383747897),
  ('X_4', 0.045944411538130364),
  ('X_9', 0.18797271945084204),
  ('X_12', 0.35160693348307526),
  ('X_1', 0.4371869